# Tidy-up Japan Typhoon Data

Tidy up Japan Typhoon data from 1951 to 2023.

# import modules

``` python
import os
from typing import List

import polars as pl
pl.Config.set_tbl_rows(7)   # limit num of lines for table preview
print('polars version', pl.__version__)
```

    polars version 0.20.9

# create df schema as a dict `{"col_name", pl.DataType}`

``` python
df_schema = {
    'h_a_indicator'               : pl.String,
    'h_b_int_num_id'              : pl.String,
    'h_c_num_data'                : pl.String,
    'h_d_tropical_cyclone_num_id' : pl.String,
    'h_e_int_num_id'              : pl.String,
    'h_f_flag_last_data_line'     : pl.String,
    'h_g_diff_hour'               : pl.String,
    'h_h_storm_name'              : pl.String,
    'h_i_date_last_rev'           : pl.String,
    'd_a_date_time'               : pl.String,
    'd_b_indicator'               : pl.String,
    'd_c_grade'                   : pl.String,
    'd_d_latitude'                : pl.String,
    'd_e_longitude'               : pl.String,
    'd_f_central_pressure_dPa'    : pl.String,
    'd_g_max_wind_speed_kt'       : pl.String,
    'd_h_dir_longest_r_50kt_wind' : pl.String,
    'd_i_longest_r_50kt_wind_nm'  : pl.String,
    'd_j_shortest_r_50kt_wind_nm' : pl.String,
    'd_k_dir_longest_r_30kt_wind' : pl.String,
    'd_l_longest_r_30kt_wind_nm'  : pl.String,
    'd_m_shortest_r_30kt_wind_nm' : pl.String,
    'd_p_landfall_or_passage'     : pl.String,
}

# print('type of df_schema:', type(df_schema))
print(df_schema)
```

    {'h_a_indicator': String, 'h_b_int_num_id': String, 'h_c_num_data': String, 'h_d_tropical_cyclone_num_id': String, 'h_e_int_num_id': String, 'h_f_flag_last_data_line': String, 'h_g_diff_hour': String, 'h_h_storm_name': String, 'h_i_date_last_rev': String, 'd_a_date_time': String, 'd_b_indicator': String, 'd_c_grade': String, 'd_d_latitude': String, 'd_e_longitude': String, 'd_f_central_pressure_dPa': String, 'd_g_max_wind_speed_kt': String, 'd_h_dir_longest_r_50kt_wind': String, 'd_i_longest_r_50kt_wind_nm': String, 'd_j_shortest_r_50kt_wind_nm': String, 'd_k_dir_longest_r_30kt_wind': String, 'd_l_longest_r_30kt_wind_nm': String, 'd_m_shortest_r_30kt_wind_nm': String, 'd_p_landfall_or_passage': String}

# create an empty dataframe

``` python
list_col_names = list(df_schema.keys())

dict_empty_data = {
    'h_a_indicator'               : [],
    'h_b_int_num_id'              : [],
    'h_c_num_data'                : [],
    'h_d_tropical_cyclone_num_id' : [],
    'h_e_int_num_id'              : [],
    'h_f_flag_last_data_line'     : [],
    'h_g_diff_hour'               : [],
    'h_h_storm_name'              : [],
    'h_i_date_last_rev'           : [],
    'd_a_date_time'               : [],
    'd_b_indicator'               : [],
    'd_c_grade'                   : [],
    'd_d_latitude'                : [],
    'd_e_longitude'               : [],
    'd_f_central_pressure_dPa'    : [],
    'd_g_max_wind_speed_kt'       : [],
    'd_h_dir_longest_r_50kt_wind' : [],
    'd_i_longest_r_50kt_wind_nm'  : [],
    'd_j_shortest_r_50kt_wind_nm' : [],
    'd_k_dir_longest_r_30kt_wind' : [],
    'd_l_longest_r_30kt_wind_nm'  : [],
    'd_m_shortest_r_30kt_wind_nm' : [],
    'd_p_landfall_or_passage'     : [],
}

df= pl.DataFrame(dict_empty_data, df_schema)
df
```

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (0, 23)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|---------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|
| str           | str            | str          | str                         | str            | str                     | str           | str            | str               | str           | str           | str       | str          | str           | str                      | str                   | str                         | str                        | str                         | str                         | str                        | str                         | str                     |

</div>

# read the data file, extract header lines and data lines

``` python
%time

# row data file path
# data_file_path = './data/RSMC_Tokyo_Typhoon_1951-2023.txt'
# data_file_path = './data/RSMC_Tokyo_Typhoon_2013-2023.txt'
# data_file_path = './data/RSMC_Tokyo_Typhoon_2019-2023.txt'
raw_data_file_path = './data/original/RSMC_Tokyo_Typhoon_2021-2023.txt'
# raw_data_file_path = './data/original/RSMC_Tokyo_Typhoon_2023.txt'

# extracted header and data file paths
extracted_header_f_path = './data/processed/header.txt'
extracted_data_f_path = './data/processed/data.txt'

# remove header and data files, if they exists
if os.path.exists(extracted_header_f_path):
    os.remove(extracted_header_f_path)

if os.path.exists(extracted_data_f_path):
    os.remove(extracted_data_f_path)

# open header and data file to write
f_header = open(extracted_header_f_path, 'w')
f_data = open(extracted_data_f_path, 'w')

# extract header lines and data lines into different files
with open(raw_data_file_path) as f:
    # read all the lines
    lines = f.readlines()

    # read each line
    for line in lines:
        # check if a line is a header/data by the length of its 1st split
        if len(line.split()[0]) == 5:
            # add header to f_header
            f_header.write(line)

        elif len(line.split()[0]) == 8:
            # add data line to f_data
            f_data.write(line)

f_header.close()
f_data.close()
```

    CPU times: total: 0 ns
    Wall time: 0 ns

# func to vstack each data record to df

``` python
def vstack_df(
    _col_names: List[str], 
    _record_items: List[str], 
    _df: pl.DataFrame,
) -> pl.DataFrame:

    # create a dict from list of col names and a list of data items
    dict_record = dict(
        zip(
            _col_names, 
            _record_items,
        )
    )

    df_to_stack = pl.DataFrame(dict_record)

    _df = _df.vstack(df_to_stack)

    return _df
```

# read header and data files to create dataframe

``` python
with open(extracted_header_f_path, 'r') as f_h:
    h_lines = f_h.readlines()

    header_cnt = 0

    f_data_line_num_start = 0
    f_data_line_num_end = -1

    for h_line in h_lines:
        # create an empty list to store header and data for each record
        headedr_items = []

        # get info from header col by col
        h_a = h_line[0:5]
        h_b = h_line[6:10]
        h_c = h_line[12:15]
        h_d = h_line[16:20]
        h_e = h_line[21:25]
        h_f = h_line[26]
        h_g = h_line[28]
        h_h = h_line[30:50].strip()
        h_i = h_line[64:72]

        # calc obsolute start and end line num in f_data to read
        # ...new start_line_num = previous end_line_num +1
        f_data_line_num_start = f_data_line_num_end + 1 
        # ...new end_line_num = new start_line_num + num_data_lines of the current data chunk
        f_data_line_num_end = f_data_line_num_start + int(h_c) - 1 

        headedr_items.extend([ 
            h_a, 
            h_b,
            h_c,
            h_d,
            h_e,
            h_f,
            h_g,
            h_h,
            h_i
        ])

        print('header:', header_cnt, '\t', headedr_items)

        # read data file by the start and end line numbers
        f_data = './data/processed/data.txt'

        with open(f_data, 'r') as f_data:
            for idx, d_line in enumerate(f_data):
                record_items = []

                if f_data_line_num_start <= idx <= f_data_line_num_end:
                    data_items = []

                    d_a = d_line[0:8].strip()
                    d_b = d_line[9:12].strip()
                    d_c = d_line[13:14].strip()
                    d_d = d_line[15:18].strip()
                    d_e = d_line[19:23].strip()
                    d_f = d_line[24:28].strip()
                    d_g = d_line[33:36].strip()
                    d_h = d_line[41].strip()
                    d_i = d_line[42:46].strip()
                    d_j = d_line[47:51].strip()
                    d_k = d_line[52].strip()
                    d_l = d_line[53:57].strip()
                    d_m = d_line[58:62].strip()
                    d_p = d_line[71].strip()

                    data_items.extend([
                        d_a,
                        d_b,
                        d_c,
                        d_d,
                        d_e,
                        d_f,
                        d_g,
                        d_h,
                        d_i,
                        d_j,
                        d_k,
                        d_l,
                        d_m,
                        d_p
                    ])

                    # join the lists of header and data items
                    record_items = headedr_items + data_items

                    # stack the df created from the current record to df
                    # option 1:
                    # dict_record = dict(zip(list_col_names, record_items))
                    # df_to_stack = pl.DataFrame(dict_record)
                    # df = df.vstack(df_to_stack)
                    # option 2:
                    df = vstack_df(list_col_names, record_items, df)

                    # # cherry-print a data line for verification
                    # if idx == 337:
                    #     print(record_items)

        header_cnt += 1
df
```

    header: 0    ['66666', '2101', '028', '0001', '2101', '0', '6', 'DUJUAN', '20210520']
    header: 1    ['66666', '2102', '073', '0002', '2102', '1', '0', 'SURIGAE', '20210715']
    header: 2    ['66666', '2103', '033', '0004', '2103', '0', '6', 'CHOI-WAN', '20210826']
    header: 3    ['66666', '2104', '011', '0005', '2104', '0', '6', 'KOGUMA', '20210826']
    header: 4    ['66666', '2105', '036', '0006', '2105', '0', '6', 'CHAMPI', '20210909']
    header: 5    ['66666', '2106', '081', '0009', '2106', '0', '6', 'IN-FA', '20211028']
    header: 6    ['66666', '2107', '033', '0010', '2107', '0', '6', 'CEMPAKA', '20211101']
    header: 7    ['66666', '2108', '037', '0011', '2108', '0', '6', 'NEPARTAK', '20211102']
    header: 8    ['66666', '2109', '056', '0014', '2109', '0', '6', 'LUPIT', '20211104']
    header: 9    ['66666', '2110', '032', '0016', '2110', '0', '6', 'MIRINAE', '20211110']
    header: 10   ['66666', '2111', '021', '0015', '2111', '0', '6', 'NIDA', '20211110']
    header: 11   ['66666', '2112', '055', '0018', '2112', '0', '6', 'OMAIS', '20211201']
    header: 12   ['66666', '2113', '034', '0020', '2113', '0', '6', 'CONSON', '20211214']
    header: 13   ['66666', '2114', '070', '0021', '2114', '0', '6', 'CHANTHU', '20211218']
    header: 14   ['66666', '2115', '017', '0022', '2115', '1', '0', 'DIANMU', '20211220']
    header: 15   ['66666', '2116', '057', '0023', '2116', '1', '0', 'MINDULLE', '20211225']
    header: 16   ['66666', '2117', '024', '0024', '2117', '0', '6', 'LIONROCK', '20220106']
    header: 17   ['66666', '2118', '031', '0025', '2118', '0', '6', 'KOMPASU', '20220106']
    header: 18   ['66666', '2119', '042', '0026', '2119', '1', '0', 'NAMTHEUN', '20220107']
    header: 19   ['66666', '2120', '039', '0027', '2120', '1', '0', 'MALOU', '20220119']
    header: 20   ['66666', '2121', '023', '0028', '2121', '0', '6', 'NYATOH', '20220204']
    header: 21   ['66666', '2122', '039', '0029', '2122', '0', '6', 'RAI', '20220204']
    header: 22   ['66666', '2201', '052', '0001', '2201', '1', '0', 'MALAKAS', '20220715']
    header: 23   ['66666', '2202', '014', '0002', '2202', '0', '6', 'MEGI', '20220716']
    header: 24   ['66666', '2203', '037', '0003', '2203', '0', '6', 'CHABA', '20221005']
    header: 25   ['66666', '2204', '043', '0004', '2204', '0', '6', 'AERE', '20221006']
    header: 26   ['66666', '2205', '024', '0006', '2205', '0', '6', 'SONGDA', '20221007']
    header: 27   ['66666', '2206', '014', '0007', '2206', '0', '6', 'TRASES', '20221101']
    header: 28   ['66666', '2207', '014', '0008', '2207', '0', '6', 'MULAN', '20221101']
    header: 29   ['66666', '2208', '035', '0009', '2208', '1', '0', 'MEARI', '20221116']
    header: 30   ['66666', '2209', '021', '0010', '2209', '0', '6', 'MA-ON', '20221116']
    header: 31   ['66666', '2210', '024', '0011', '2210', '1', '0', 'TOKAGE', '20221120']
    header: 32   ['66666', '2211', '069', '0013', '2211', '1', '0', 'HINNAMNOR', '20221124']
    header: 33   ['66666', '2212', '063', '0016', '2212', '0', '6', 'MUIFA', '20221124']
    header: 34   ['66666', '2213', '026', '0017', '2213', '1', '0', 'MERBOK', '20221125']
    header: 35   ['66666', '2214', '047', '0018', '2214', '0', '6', 'NANMADOL', '20221219']
    header: 36   ['66666', '2215', '028', '0019', '2215', '0', '6', 'TALAS', '20221223']
    header: 37   ['66666', '2216', '033', '0020', '2216', '0', '6', 'NORU', '20221226']
    header: 38   ['66666', '2217', '028', '0021', '2217', '1', '0', 'KULAP', '20230104']
    header: 39   ['66666', '2218', '032', '0022', '2218', '0', '6', 'ROKE', '20230104']
    header: 40   ['66666', '2219', '008', '0023', '2219', '0', '6', 'SONCA', '20230104']
    header: 41   ['66666', '2220', '024', '0025', '2220', '0', '6', 'NESAT', '20230104']
    header: 42   ['66666', '2221', '015', '0026', '2221', '1', '0', 'HAITANG', '20230112']
    header: 43   ['66666', '2222', '033', '0028', '2222', '0', '6', 'NALGAE', '20230112']
    header: 44   ['66666', '2223', '024', '0029', '2223', '0', '6', 'BANYAN', '20230117']
    header: 45   ['66666', '2224', '014', '0030', '2224', '0', '6', 'YAMANEKO', '20230119']
    header: 46   ['66666', '2225', '011', '0031', '2225', '0', '6', 'PAKHAR', '20230123']
    header: 47   ['66666', '2301', '014', '0003', '2301', '0', '6', 'SANVU', '20230704']
    header: 48   ['66666', '2302', '067', '0004', '2302', '0', '6', 'MAWAR', '20230830']
    header: 49   ['66666', '2303', '044', '0005', '2303', '1', '0', 'GUCHOL', '20230906']
    header: 50   ['66666', '2304', '023', '0006', '2304', '0', '6', 'TALIM', '20231025']
    header: 51   ['66666', '2305', '042', '0007', '2305', '0', '6', 'DOKSURI', '20231025']
    header: 52   ['66666', '2306', '106', '0008', '2306', '0', '6', 'KHANUN', '20231027']
    header: 53   ['66666', '2307', '061', '0009', '2307', '0', '6', 'LAN', '20231114']
    header: 54   ['66666', '2308', '042', '0010', '2308', '1', '0', 'DORA', '20231115']
    header: 55   ['66666', '2309', '051', '0011', '2309', '0', '6', 'SAOLA', '20231127']
    header: 56   ['66666', '2310', '028', '0012', '2310', '1', '0', 'DAMREY', '20231127']
    header: 57   ['66666', '2311', '040', '0013', '2311', '0', '6', 'HAIKUI', '20231127']
    header: 58   ['66666', '2312', '032', '0014', '2312', '0', '6', 'KIROGI', '20231128']
    header: 59   ['66666', '2313', '019', '0015', '2313', '0', '6', 'YUN-YEUNG', '20231129']
    header: 60   ['66666', '2314', '046', '0016', '2314', '0', '6', 'KOINU', '20240110']
    header: 61   ['66666', '2315', '039', '0017', '2315', '1', '0', 'BOLAVEN', '20240110']
    header: 62   ['66666', '2316', '013', '0018', '2316', '0', '6', 'SANBA', '20240111']

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (2_272, 23)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|---------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|
| str           | str            | str          | str                         | str            | str                     | str           | str            | str               | str           | str           | str       | str          | str           | str                      | str                   | str                         | str                        | str                         | str                         | str                        | str                         | str                     |
| "66666"       | "2101"         | "028"        | "0001"                      | "2101"         | "0"                     | "6"           | "DUJUAN"       | "20210520"        | "21021606"    | "002"         | "2"       | "069"        | "1369"        | "1004"                   | "000"                 | ""                          | ""                         | ""                          | ""                          | ""                         | ""                          | ""                      |
| "66666"       | "2101"         | "028"        | "0001"                      | "2101"         | "0"                     | "6"           | "DUJUAN"       | "20210520"        | "21021612"    | "002"         | "2"       | "069"        | "1362"        | "1006"                   | "000"                 | ""                          | ""                         | ""                          | ""                          | ""                         | ""                          | ""                      |
| "66666"       | "2101"         | "028"        | "0001"                      | "2101"         | "0"                     | "6"           | "DUJUAN"       | "20210520"        | "21021618"    | "002"         | "2"       | "068"        | "1351"        | "1004"                   | "000"                 | ""                          | ""                         | ""                          | ""                          | ""                         | ""                          | ""                      |
| …             | …              | …            | …                           | …              | …                       | …             | …              | …                 | …             | …             | …         | …            | …             | …                        | …                     | …                           | …                          | …                           | …                           | …                          | …                           | …                       |
| "66666"       | "2316"         | "013"        | "0018"                      | "2316"         | "0"                     | "6"           | "SANBA"        | "20240111"        | "23101918"    | "002"         | "3"       | "214"        | "1095"        | "1002"                   | "035"                 | "0"                         | "0000"                     | "0000"                      | "9"                         | "0150"                     | "0150"                      | ""                      |
| "66666"       | "2316"         | "013"        | "0018"                      | "2316"         | "0"                     | "6"           | "SANBA"        | "20240111"        | "23102000"    | "002"         | "2"       | "214"        | "1098"        | "1008"                   | "000"                 | ""                          | ""                         | ""                          | ""                          | ""                         | ""                          | ""                      |
| "66666"       | "2316"         | "013"        | "0018"                      | "2316"         | "0"                     | "6"           | "SANBA"        | "20240111"        | "23102006"    | "002"         | "2"       | "209"        | "1097"        | "1008"                   | "000"                 | ""                          | ""                         | ""                          | ""                          | ""                         | ""                          | ""                      |

</div>

# replace empty string with ‘null’

``` python
df = df.select(
    # pl.when(pl.col(pl.Utf8).str.lengths()==0) # lengths() deprecated
    #   .then(None)
    #   .otherwise(pl.col(pl.Utf8))             # pl.Utf8 replaced by pl.String
    #   .keep_name()                            # .keep_name() deprecated

    pl.when(pl.col(pl.Utf8).str.len_bytes()==0)
      .then(None)
      .otherwise(pl.col(pl.String))
      .name.keep()
)
df
```

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (2_272, 23)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|---------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|
| str           | str            | str          | str                         | str            | str                     | str           | str            | str               | str           | str           | str       | str          | str           | str                      | str                   | str                         | str                        | str                         | str                         | str                        | str                         | str                     |
| "66666"       | "2101"         | "028"        | "0001"                      | "2101"         | "0"                     | "6"           | "DUJUAN"       | "20210520"        | "21021606"    | "002"         | "2"       | "069"        | "1369"        | "1004"                   | "000"                 | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| "66666"       | "2101"         | "028"        | "0001"                      | "2101"         | "0"                     | "6"           | "DUJUAN"       | "20210520"        | "21021612"    | "002"         | "2"       | "069"        | "1362"        | "1006"                   | "000"                 | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| "66666"       | "2101"         | "028"        | "0001"                      | "2101"         | "0"                     | "6"           | "DUJUAN"       | "20210520"        | "21021618"    | "002"         | "2"       | "068"        | "1351"        | "1004"                   | "000"                 | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| …             | …              | …            | …                           | …              | …                       | …             | …              | …                 | …             | …             | …         | …            | …             | …                        | …                     | …                           | …                          | …                           | …                           | …                          | …                           | …                       |
| "66666"       | "2316"         | "013"        | "0018"                      | "2316"         | "0"                     | "6"           | "SANBA"        | "20240111"        | "23101918"    | "002"         | "3"       | "214"        | "1095"        | "1002"                   | "035"                 | "0"                         | "0000"                     | "0000"                      | "9"                         | "0150"                     | "0150"                      | null                    |
| "66666"       | "2316"         | "013"        | "0018"                      | "2316"         | "0"                     | "6"           | "SANBA"        | "20240111"        | "23102000"    | "002"         | "2"       | "214"        | "1098"        | "1008"                   | "000"                 | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| "66666"       | "2316"         | "013"        | "0018"                      | "2316"         | "0"                     | "6"           | "SANBA"        | "20240111"        | "23102006"    | "002"         | "2"       | "209"        | "1097"        | "1008"                   | "000"                 | null                        | null                       | null                        | null                        | null                       | null                        | null                    |

</div>

# cast data type for columns

``` python
df = df.with_columns(
    pl.col('h_a_indicator').cast(pl.Int32),
    pl.col('h_b_int_num_id').cast(pl.Int16),
    pl.col('h_c_num_data').cast(pl.Int16),
    pl.col('h_d_tropical_cyclone_num_id').cast(pl.Int8),
    pl.col('h_e_int_num_id').cast(pl.Int16),
    pl.col('h_f_flag_last_data_line').cast(pl.Int8),
    pl.col('h_g_diff_hour').cast(pl.Int8),
    pl.col('d_b_indicator').cast(pl.Int8),
    pl.col('d_c_grade').cast(pl.Int8),
    pl.col('d_d_latitude').cast(pl.Float64),
    pl.col('d_e_longitude').cast(pl.Float64),
    pl.col('d_f_central_pressure_dPa').cast(pl.Int16),
    pl.col('d_g_max_wind_speed_kt').cast(pl.Int16),
    pl.col('d_h_dir_longest_r_50kt_wind').cast(pl.Int16),
    pl.col('d_i_longest_r_50kt_wind_nm').cast(pl.Int16),
    pl.col('d_j_shortest_r_50kt_wind_nm').cast(pl.Int16),
    pl.col('d_k_dir_longest_r_30kt_wind').cast(pl.Int16),
    pl.col('d_l_longest_r_30kt_wind_nm').cast(pl.Int16),
    pl.col('d_m_shortest_r_30kt_wind_nm').cast(pl.Int16),
)
df
```

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (2_272, 23)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|---------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|
| i32           | i16            | i16          | i8                          | i16            | i8                      | i8            | str            | str               | str           | i8            | i8        | f64          | f64           | i16                      | i16                   | i16                         | i16                        | i16                         | i16                         | i16                        | i16                         | str                     |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "21021606"    | 2             | 2         | 69.0         | 1369.0        | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "21021612"    | 2             | 2         | 69.0         | 1362.0        | 1006                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "21021618"    | 2             | 2         | 68.0         | 1351.0        | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| …             | …              | …            | …                           | …              | …                       | …             | …              | …                 | …             | …             | …         | …            | …             | …                        | …                     | …                           | …                          | …                           | …                           | …                          | …                           | …                       |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "23101918"    | 2             | 3         | 214.0        | 1095.0        | 1002                     | 35                    | 0                           | 0                          | 0                           | 9                           | 150                        | 150                         | null                    |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "23102000"    | 2             | 2         | 214.0        | 1098.0        | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "23102006"    | 2             | 2         | 209.0        | 1097.0        | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |

</div>

# add 4 `0` @ end of each item in `d_a_date_time` column

``` python
df =  df.with_columns(
    pl.col('d_a_date_time').str.pad_end(12, "0")
)
df
```

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (2_272, 23)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time  | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|----------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|
| i32           | i16            | i16          | i8                          | i16            | i8                      | i8            | str            | str               | str            | i8            | i8        | f64          | f64           | i16                      | i16                   | i16                         | i16                        | i16                         | i16                         | i16                        | i16                         | str                     |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216060000" | 2             | 2         | 69.0         | 1369.0        | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216120000" | 2             | 2         | 69.0         | 1362.0        | 1006                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216180000" | 2             | 2         | 68.0         | 1351.0        | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| …             | …              | …            | …                           | …              | …                       | …             | …              | …                 | …              | …             | …         | …            | …             | …                        | …                     | …                           | …                          | …                           | …                           | …                          | …                           | …                       |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231019180000" | 2             | 3         | 214.0        | 1095.0        | 1002                     | 35                    | 0                           | 0                          | 0                           | 9                           | 150                        | 150                         | null                    |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231020000000" | 2             | 2         | 214.0        | 1098.0        | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231020060000" | 2             | 2         | 209.0        | 1097.0        | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    |

</div>

# add datetime column

``` python
df = df.with_columns(
    # pl.col('h_i_date_last_rev')
    #   .str.strptime(pl.Date, format="%Y%m%d")
    #   .alias('date_last_rev'),
    pl.col('d_a_date_time')
      .str.strptime(pl.Datetime, format="%y%m%d%H%M%S")
      .alias('date_time'),
)
df
```

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (2_272, 24)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time  | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage | date_time           |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|----------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|---------------------|
| i32           | i16            | i16          | i8                          | i16            | i8                      | i8            | str            | str               | str            | i8            | i8        | f64          | f64           | i16                      | i16                   | i16                         | i16                        | i16                         | i16                         | i16                        | i16                         | str                     | datetime\[μs\]      |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216060000" | 2             | 2         | 69.0         | 1369.0        | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2021-02-16 06:00:00 |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216120000" | 2             | 2         | 69.0         | 1362.0        | 1006                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2021-02-16 12:00:00 |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216180000" | 2             | 2         | 68.0         | 1351.0        | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2021-02-16 18:00:00 |
| …             | …              | …            | …                           | …              | …                       | …             | …              | …                 | …              | …             | …         | …            | …             | …                        | …                     | …                           | …                          | …                           | …                           | …                          | …                           | …                       | …                   |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231019180000" | 2             | 3         | 214.0        | 1095.0        | 1002                     | 35                    | 0                           | 0                          | 0                           | 9                           | 150                        | 150                         | null                    | 2023-10-19 18:00:00 |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231020000000" | 2             | 2         | 214.0        | 1098.0        | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2023-10-20 00:00:00 |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231020060000" | 2             | 2         | 209.0        | 1097.0        | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2023-10-20 06:00:00 |

</div>

# adjust lat and lon value

``` python
df = df.with_columns(
    pl.col("d_d_latitude").mul(0.1), 
    pl.col("d_e_longitude").mul(0.1)
)
df
```

<div><style>
.dataframe > thead > tr,
.dataframe > tbody > tr {
  text-align: right;
  white-space: pre-wrap;
}
</style>
<small>shape: (2_272, 24)</small>

| h_a_indicator | h_b_int_num_id | h_c_num_data | h_d_tropical_cyclone_num_id | h_e_int_num_id | h_f_flag_last_data_line | h_g_diff_hour | h_h_storm_name | h_i_date_last_rev | d_a_date_time  | d_b_indicator | d_c_grade | d_d_latitude | d_e_longitude | d_f_central_pressure_dPa | d_g_max_wind_speed_kt | d_h_dir_longest_r_50kt_wind | d_i_longest_r_50kt_wind_nm | d_j_shortest_r_50kt_wind_nm | d_k_dir_longest_r_30kt_wind | d_l_longest_r_30kt_wind_nm | d_m_shortest_r_30kt_wind_nm | d_p_landfall_or_passage | date_time           |
|---------------|----------------|--------------|-----------------------------|----------------|-------------------------|---------------|----------------|-------------------|----------------|---------------|-----------|--------------|---------------|--------------------------|-----------------------|-----------------------------|----------------------------|-----------------------------|-----------------------------|----------------------------|-----------------------------|-------------------------|---------------------|
| i32           | i16            | i16          | i8                          | i16            | i8                      | i8            | str            | str               | str            | i8            | i8        | f64          | f64           | i16                      | i16                   | i16                         | i16                        | i16                         | i16                         | i16                        | i16                         | str                     | datetime\[μs\]      |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216060000" | 2             | 2         | 6.9          | 136.9         | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2021-02-16 06:00:00 |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216120000" | 2             | 2         | 6.9          | 136.2         | 1006                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2021-02-16 12:00:00 |
| 66666         | 2101           | 28           | 1                           | 2101           | 0                       | 6             | "DUJUAN"       | "20210520"        | "210216180000" | 2             | 2         | 6.8          | 135.1         | 1004                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2021-02-16 18:00:00 |
| …             | …              | …            | …                           | …              | …                       | …             | …              | …                 | …              | …             | …         | …            | …             | …                        | …                     | …                           | …                          | …                           | …                           | …                          | …                           | …                       | …                   |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231019180000" | 2             | 3         | 21.4         | 109.5         | 1002                     | 35                    | 0                           | 0                          | 0                           | 9                           | 150                        | 150                         | null                    | 2023-10-19 18:00:00 |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231020000000" | 2             | 2         | 21.4         | 109.8         | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2023-10-20 00:00:00 |
| 66666         | 2316           | 13           | 18                          | 2316           | 0                       | 6             | "SANBA"        | "20240111"        | "231020060000" | 2             | 2         | 20.9         | 109.7         | 1008                     | 0                     | null                        | null                       | null                        | null                        | null                       | null                        | null                    | 2023-10-20 06:00:00 |

</div>

# check result

``` python
# df.describe()
```

# write df as parquet

``` python
df_parquet_f_path = "./data/processed/df.parquet"

# remove parquet file, if it exists
if os.path.exists(df_parquet_f_path):
    os.remove(df_parquet_f_path)

df.write_parquet(df_parquet_f_path)
print('df written as parquet file')
```

    df written as parquet file

# create map

``` python
import plotly.express as px

df = pl.read_parquet(df_parquet_f_path)
print('read df as parquet')

fig = px.line_mapbox(
    df,
    # lat=df["d_d_latitude"],
    # lon=df["d_e_longitude"],
    # color=df["h_b_int_num_id"],
    lat="d_d_latitude",
    lon="d_e_longitude",
    color="h_b_int_num_id",
    zoom=3,
    height=1000,
    # animation_frame="h_b_int_num_id"
)

fig.update_layout(
    # mapbox_style="open-street-map",
    mapbox_style="carto-darkmatter",
    # mapbox_zoom=4,
    mapbox_center_lat=36,
    margin={"r":0,"t":0,"l":0,"b":0},
)

fig.show()
```